## Day 7: The Treachery of Whales

We moeten het getal vinden met gelijke aantallen hoger en lager. dat is de median

In [54]:
import statistics
from pathlib import Path
import re
from typing import List, Dict, Tuple

In [2]:
day7_test = [int(x) for x in '16,1,2,0,4,2,7,1,2,14'.split(',')]

In [3]:
statistics.median_low(day7_test), statistics.median_high(day7_test)

(2, 2)

In [4]:
m = statistics.median_low(day7_test)
sum(abs(x-m) for x in day7_test)

37

In [5]:
day7_data = [int(x) for x in Path('7-data.txt').read_text().split(',')]

In [6]:
m = statistics.median_low(day7_data)
sum(abs(x-m) for x in day7_data)

343468

## Part 2: more fuel

As it turns out, crab submarine engines don't burn fuel at a constant rate. Instead, each change of 1 step in horizontal position costs 1 more unit of fuel than the last: the first step costs 1, the second step costs 2, the third step costs 3, and so on.

Hier gebruiken we gemiddelde. Blijkbaar moeten we naar beneden afronden. waarom?  
vermoedelijk omdat de median lager is.

In [8]:
m = round(statistics.mean(day7_test))
m, sum(sum(range(abs(x-m)+1)) for x in day7_test)

(5, 168)

In [11]:
m = round(statistics.mean(day7_data))
statistics.mean(day7_data), m, sum(sum(range(abs(x-m)+1)) for x in day7_data)

(478.568, 479, 96086306)

In [12]:
m = int(statistics.mean(day7_data))
statistics.mean(day7_data), m, sum(sum(range(abs(x-m)+1)) for x in day7_data)

(478.568, 478, 96086265)

In [ ]:
m = round(statistics.mean(day7_data))
statistics.mean(day7_data), m, sum(sum(range(abs(x-m)+1)) for x in day7_data)

(478.568, 479, 96086306)

## Day 8: Seven Segment Search

In [135]:
class Display():
    def __init__(self, data: str) -> None:
        signal, output = data.split(" | ")
        self.signal = re.findall(r'\w+', signal)
        self.output = re.findall(r'\w+', output)

    def __str__(self) -> str:
        return f"{self.signal} | {self.output}"

    @property
    def all_signals(self):
        return self.signal + self.output

print(Display("be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe"))

['be', 'cfbegad', 'cbdgef', 'fgaecd', 'cgeb', 'fdcge', 'agebfd', 'fecdb', 'fabcd', 'edb'] | ['fdgacbe', 'cefdb', 'cefbgd', 'gcbe']


In [137]:
day8_test = [Display(l) for l in Path('8-test.txt').read_text().splitlines()]

In [138]:
len([x for d in day8_test for x in d.output if len(x) in (2, 4, 3, 7)])

26

In [139]:
day8_data = [Display(l) for l in Path('8-data.txt').read_text().splitlines()]
len([x for d in day8_data for x in d.output if len(x) in (2, 4, 3, 7)])

548

## Day 8: Part 2, Mapping

In [93]:
from collections import OrderedDict
display_map = OrderedDict({0:"abcefg", 1:"cf", 2: "acdeg", 3: "acdfg", 4:"bcdf", 5: "abdfg", 6: "abdefg", 7: "acf", 8: "abcdefg", 9: "abcdfg"})
signal_map = {v: k for k,v in display_map.items()}

In [142]:
import itertools
import logging

def valid_map(options: Dict[str, List[str]], map: Dict[str, str]) -> bool:
    reverse_map = {v: k for k,v in map.items()}
    translated = {k: ["".join(sorted(i.translate(str.maketrans(reverse_map))))
                      for i in v] for k, v in options.items()}
    rv = (False not in (x in o for x, o in translated.items()))
    logging.debug("%s %s %s", map, translated, rv)
    return rv, translated


def get_options(data: List[str]) -> Dict[str, List[str]]:
    options = {n:[] for n in display_map.values()}
    for v in display_map.values():
        for signal in data:
            if len(v) == len(signal):
                options[v].append(signal)
    return options

def signal_options(data: Dict[str, List[str]]) -> Dict[str, List[str]]:
    options = OrderedDict({s: set('abcdefg') for s in "abcdefg"})
    for output, signals in data.items():
        possible_wires = set("".join(signals))
        for wire in output:
            options[wire] = options[wire].intersection(possible_wires)
    return options

def find_map(data: List[str]) -> Dict[str, int]:
    options = get_options(data)
    signals = signal_options(options)
    map_options = [dict(zip(signals.keys(), x)) for x in itertools.product(*signals.values()) if len(set(x))==7]
    for map in map_options:
        if valid_map(options, map)[0]:
            return map

def decode(signal: str, map: dict):
    reverse_map = {v: k for k,v in map.items()}
    k = "".join(sorted(signal.translate(str.maketrans(reverse_map))))
    return k, signal_map[k]

def decode_output(display: Display) -> int:
    map = find_map(display.all_signals)
    return int("".join(str(decode(s, map)[1]) for s in display.output))
    


In [133]:
logging.getLogger().setLevel(level='INFO')
test_signal = Display('acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf')
map = find_map(test_signal.signal + test_signal.output)
print(map)
test_reference = {'a': 'd', 'b': 'e', 'c': 'a', 'd': 'f', 'e': 'g', 'f': 'b', 'g': 'c'}
assert map == test_reference
for o in test_signal.output:
    logging.debug(o)
    print(o, decode(o, map))
print(decode_output(test_signal.output, map))

{'a': 'd', 'b': 'e', 'c': 'a', 'd': 'f', 'e': 'g', 'f': 'b', 'g': 'c'}
cdfeb ('abdfg', 5)
fcadb ('acdfg', 3)
cdfeb ('abdfg', 5)
cdbaf ('acdfg', 3)
5353


In [72]:
get_options(day8_test[0].signal + day8_test[0].output)

{'abcefg': ['cbdgef', 'fgaecd', 'agebfd', 'cefbgd'],
 'cf': ['be'],
 'acdeg': ['fdcge', 'fecdb', 'fabcd', 'cefdb'],
 'acdfg': ['fdcge', 'fecdb', 'fabcd', 'cefdb'],
 'bcdf': ['cgeb', 'gcbe'],
 'abdfg': ['fdcge', 'fecdb', 'fabcd', 'cefdb'],
 'abdefg': ['cbdgef', 'fgaecd', 'agebfd', 'cefbgd'],
 'acf': ['edb'],
 'abcdefg': ['cfbegad', 'fdgacbe'],
 'abcdfg': ['cbdgef', 'fgaecd', 'agebfd', 'cefbgd']}

### Test data
Following this same process for each entry in the second, larger example above, the output value of each entry can be determined:

```
fdgacbe cefdb cefbgd gcbe: 8394
fcgedb cgb dgebacf gc: 9781
cg cg fdcagb cbg: 1197
efabcd cedba gadfec cb: 9361
gecf egdcabf bgf bfgea: 4873
gebdcfa ecba ca fadegcb: 8418
cefg dcbef fcge gbcadfe: 4548
ed bcgafe cdgba cbgef: 1625
gbdfcae bgc cg cgb: 8717
fgae cfgab fg bagce: 4315
```

Adding all of the output values in this larger example produces `61229`.

In [145]:
for d in day8_test:
    print(decode_output(d))

assert sum(decode_output(x) for x in day8_test) == 61229

8394
9781
1197
9361
4873
8418
4548
1625
8717
4315


In [146]:
sum(decode_output(x) for x in day8_data)

1074888